从Adidas中国官网爬取所有线下店的位置、类型等数据。[其官网](https://www.adidas.com.cn/location/storefinder?locale=zh_CN),它是用json穿搜索数据的，所有比较容易的，不过需要主动设置Cookie，否则是会被forbidden的。

In [3]:
import json
import time
import requests
import traceback
from lxml import etree

### 爬取

In [4]:
province={130000: "河北省",
        540000: "西藏",
        330000: "浙江省",
        620000: "甘肃省",
        360000: "江西省",
        210000: "辽宁省",
        500000: "重庆",
        370000: "山东省",
        460000: "海南省",
        530000: "云南省",
        520000: "贵州省",
        340000: "安徽省",
        220000: "吉林省",
        630000: "青海省",
        510000: "四川省",
        150000: "内蒙古",
        420000: "湖北省",
        640000: "宁夏",
        450000: "广西",
        650000: "新疆",
        310000: "上海",
        230000: "黑龙江省",
        410000: "河南省",
        140000: "山西省",
        110000: "北京",
        440000: "广东省",
        320000: "江苏省",
        120000: "天津",
        430000: "湖南省",
        610000: "陕西省",
        350000: "福建省"} 
#这部分没找到json，直接从https://img.adidas.com.cn/??libs/spice/js/jquery.spice.tools.js里抄

In [5]:
def getOneUrl(url,w=0): #获取一个页面
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36', 
                   'Cookie': 'JSESSIONID=A4916E70EDC6F89BE6D9925544E9B08C-n51; _ga=GA1.3.102630160.1556376306; _gid=GA1.3.353691624.1556376306; XSRF-TOKEN=6af9bf7f-9fcb-4e17-a2b0-5ea533870bd5; adidas_recommend_cookie_key=%u62A2YEEZY%2CClima%2CNite%20JOGGER%2CGame%20of%20Thrones%2Cultraboost%2Cyung%2096%2Csuperstar%2Cstan%20smith%2C%u4E09%u53F6%u8349%2CNMD; s_c_c_n=0; _Jo0OQK=74552BCA648073AE899B28A0D522495E51EAB059F5523F1D374782338C04ED87FAE8FB1E8F487E7835F55EA6C8AE8086E88B7899FF054733A798F56662BF2EEBDE7CD70587E0A40CF3EE1B24C75C2FEBCEFE1B24C75C2FEBCEF61332EE600EB2D15FA5A47021B17D55BGJ1Z1LA==; pt_ref_7f28852c=; pt_s_7f28852c=vt=1556384159877&cad=; pt_7f28852c=uid=2sXBoRWY1OVmX3urOjQavQ&nid=0&vid=RbuqYYxba1lXaOrzBZtR4Q&vn=3&pvn=1&sact=1556385728025&to_flag=1&pl=udUDxHiiaEhlGzijk/Jvrw*pt*1556385669042'}
        r=requests.get(url,headers=headers,timeout=30)
        #r.encoding=r.apparent_encoding
        return r.text
    except:
        
        try:
            print(str(r),r.text)
        except:
            print('ccc',w,url)
        #print(str(r),r.text)
        traceback.print_exc()
        if w>3:
            return None
        else:
            time.sleep(w)
            w+=1
            txt=getOneUrl(url,w)
            if txt==None:
                print(i,url) #continue
            else:
                return txt
        return None

def getStoreTypes(city):#input: a city output alist
    stlst=[]
    url='https://www.adidas.com.cn/location/storetype.json?city={0}&_=1556377484672'.format(c)
    rtxt=getOneUrl(url)
    if rtxt==None:
        print('rtxt==None',city,url)
        rtxt='[]'
    sts=json.loads(rtxt)
    #like:[{"id":0,"storeType":"运动休闲概念店","storeTypeEN":"Sportswear Collective (SWC)"},{"id":0,"storeType":"运动表现店","storeTypeEN":"Sport Performance"},{"id":0,"storeType":"折扣店","storeTypeEN":"Outlet"}]
    for sd in sts: #sotretypes
        stlst.append(sd['storeType'])
    return stlst
  
def getSTotal(pv,c,store):
    url='https://www.adidas.com.cn/location/storelist.json?startPage=1&size=12&province={0}&city={1}&storeName=&types%5B%5D={2}&_=1556376900755'.format(pv,c,store)
    rtxt=getOneUrl(url)
    if rtxt==None:
        print('getSTotal==None',pv,c,url)
        stt=''
    stt=json.loads(rtxt)
    return [stt['storeTotal'],stt['storeMapResult']]

#https://www.adidas.com.cn/location/storelist.json?startPage=1&size=12&province=&city=%E5%8C%97%E4%BA%AC%E5%B8%82&storeName=&_=1556390707883

In [6]:
res={}
klst=[]

In [11]:
cc=0
for pkey,pval in province.items():
    prtt=['','','','']
    try:
        cc+=1
        pv=pval.replace('省','')
        prtt[0]=pv
        print(pv,cc,end=' ')
        res[pv]={}
        purl='https://www.adidas.com.cn/location/city.json?province={0}&_=1556377173555'.format(pv)
        cts=getOneUrl(purl)
        if cts==None:
            print('cts==None',pv,purl)
            cts='[]'
        citys=json.loads(cts)
        for c in citys:
            res[pv][c]={}
            prtt[1]=c
            sts=getStoreTypes(c) #sotretypes list str
            for s in sts:
                prtt[2]=s
                stotal=getSTotal(pv,c,s)
                res[pv][c][s]=stotal[0]
                klst.extend(stotal[1])
                prtt[3]=stotal[0]
                #time.sleep(1)
            time.sleep(1)
            
    except:
        traceback.print_exc()
        print(prtt)
        #break


河北 1 西藏 2 浙江 3 甘肃 4 江西 5 辽宁 6 ccc https://www.adidas.com.cn/location/storetype.json?city=新民市&_=1556377484672


Traceback (most recent call last):
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/lyndonlcf/anaconda3/lib/python3.6/socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 346, in _make_request
    self._validate_conn(conn)
  File 

ccc https://www.adidas.com.cn/location/storetype.json?city=新民市&_=1556377484672


Traceback (most recent call last):
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/lyndonlcf/anaconda3/lib/python3.6/socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 346, in _make_request
    self._validate_conn(conn)
  File 

重庆 7 山东 8 海南 9 云南 10 贵州 11 安徽 12 吉林 13 青海 14 四川 15 内蒙古 16 湖北 17 宁夏 18 广西 19 新疆 20 上海 21 黑龙江 22 河南 23 山西 24 北京 25 广东 26 ccc https://www.adidas.com.cn/location/storetype.json?city=阳东县&_=1556377484672


...

Traceback (most recent call last):
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/lyndonlcf/anaconda3/lib/python3.6/socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 346, in _make_request
    self._validate_conn(conn)
  File 

ccc https://www.adidas.com.cn/location/storelist.json?startPage=1&size=12&province=福建&city=泉州市&storeName=&types%5B%5D=童装店&_=1556376900755


Traceback (most recent call last):
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/lyndonlcf/anaconda3/lib/python3.6/socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 346, in _make_request
    self._validate_conn(conn)
  File 

ccc https://www.adidas.com.cn/location/storelist.json?startPage=1&size=12&province=福建&city=泉州市&storeName=&types%5B%5D=童装店&_=1556376900755


Traceback (most recent call last):
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/lyndonlcf/anaconda3/lib/python3.6/socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -2] Name or service not known

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 601, in urlopen
    chunked=chunked)
  File "/home/lyndonlcf/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 346, in _make_request
    self._validate_conn(conn)
  File 

In [ ]:
#https://www.adidas.com.cn/location/storefinder?locale=zh_CN

In [8]:
res

{'河北': {'安国市': {'运动表现店': 1},
  '安平县': {'运动表现店': 1},
  '安新县': {'运动表现店': 2, '运动休闲店': 1}}}

In [12]:
print(len(klst))  #需要去重
klst[0]  

10101


{'addressCN': '河北安国市药华大街170号宜家旺商场4楼',
 'addressEN': '河北安国市药华大街170号宜家旺商场4楼',
 'city': '安国市',
 'id': 5351,
 'latitude': '38.422413',
 'longitude': '115.321785',
 'pOSNameCN': '安国市药华大街宜家旺商场运动表现店',
 'pOSNameEN': '安国市药华大街宜家旺商场运动表现店',
 'province': '河北',
 'storeType': '运动表现店',
 'storeTypeEN': 'Sport Performance',
 'telephone': '18832238828',
 'workHourCN': '',
 'workHourEn': ''}

In [6]:
res

{'河北': {'安国市': {'运动表现店': 1},
  '安平县': {'运动表现店': 1},
  '安新县': {'运动表现店': 3, '运动休闲店': 3},
  '霸州市': {'运动表现店': 4},
  '保定市': {'运动表现店': 30, '三叶草店': 30, '运动休闲店': 30, '童装店': 30, '折扣店': 30},
  '泊头市': {'运动表现店': 3, '运动休闲店': 3},
  '沧州市': {'运动表现店': 14, '运动休闲店': 14, '童装店': 14},
  '昌黎县': {'运动表现店': 2},
  '承德市': {'运动表现店': 12, '运动休闲店': 12, '童装店': 12},
  '磁县': {'运动表现店': 3, '运动休闲店': 3},
  '大城县': {'运动表现店': 3, '运动休闲店': 3},
  '大名县': {'运动表现店': 1},
  '定兴县': {'运动表现店': 2},
  '定州市': {'运动表现店': 8, '运动休闲店': 8},
  '东光县': {'运动表现店': 4, '运动休闲店': 4},
  '丰宁满族自治县': {'运动表现店': 1},
  '抚宁县': {'运动表现店': 1},
  '阜城县': {'运动表现店': 1},
  '高碑店市': {'运动表现店': 8, '运动休闲店': 8},
  '高阳县': {'运动表现店': 2, '运动休闲店': 2},
  '故城县': {'运动表现店': 1},
  '固安县': {'运动表现店': 3},
  '馆陶县': {'运动表现店': 1},
  '广宗县': {'运动表现店': 1},
  '海兴县': {'运动表现店': 1},
  '邯郸市': {'运动表现店': 18, '三叶草店': 18, '运动休闲店': 18, '童装店': 18, '折扣店': 18},
  '河间市': {'运动表现店': 3, '运动休闲店': 3},
  '衡水市': {'运动表现店': 11, '运动休闲店': 11, '童装店': 11},
  '怀来县': {'运动表现店': 3, '运动休闲店': 3},
  '黄骅市': {'运动表现店': 10, '运动休闲店': 

### 数据整理与输出

In [7]:
with open('adidas_txt00.txt','w') as f:
    f.write(str(res))

In [8]:
len(res)

31

In [ ]:
len(province)

In [13]:
cck=0
for ki,vi in res.items():
    for kj,vj in vi.items():
        for k,v in vj.items():
            cck+=v
            

In [14]:
cck #一共多少家店

10096

In [15]:
sww=[]
for ki,vi in res.items():
    for kj,vj in vi.items():
        for k,v in vj.items():
            sww.append({'province':ki,'city':kj,'stype':k,'stcount':v})
len(sww)

2572

In [ ]:
cck

In [16]:
import pandas as pd

In [17]:
sdf=pd.DataFrame(sww)
sdf.head(2)

,city,province,stcount,stype
0,安国市,河北,1,运动表现店
1,安平县,河北,1,运动表现店


In [18]:
sdf.to_excel('./adidas/adidas-表现店统计.xlsx')

In [19]:
kdf=pd.DataFrame(klst)
kdf.to_excel('./adidas/adidas-表现店详情.xlsx')

In [20]:
kdf.head(3)

,addressCN,addressEN,city,id,latitude,longitude,pOSNameCN,pOSNameEN,province,storeType,storeTypeEN,telephone,workHourCN,workHourEn
0,河北安国市药华大街170号宜家旺商场4楼,河北安国市药华大街170号宜家旺商场4楼,安国市,5351,38.422413,115.321785,安国市药华大街宜家旺商场运动表现店,安国市药华大街宜家旺商场运动表现店,河北,运动表现店,Sport Performance,18832238828,,
1,河北安平县为民街22号瑞泰广场2楼,河北安平县为民街22号瑞泰广场2楼,安平县,6203,38.233459,115.52456,安平县为民街瑞泰广场运动表现店,安平县为民街瑞泰广场运动表现店,河北,运动表现店,Sport Performance,15076826658,,
2,河北安新县新南大街31号华傲购物中心1楼,河北安新县新南大街31号华傲购物中心1楼,安新县,4407,38.9183262,115.93166742,安新县新南大街31号运动表现店,安新县新南大街31号运动表现店,河北,运动表现店,Sport Performance,13833092206,,


In [21]:
len(kdf)

10101

In [13]:
for ki,vi in res.items():
    df=pd.DataFrame(vi)
    df.to_excel('./adidas/adidas-{0}.xlsx'.format(ki),sheet_name=str(ki))
    print(ki,end=' ')

河北 西藏 浙江 甘肃 江西 辽宁 重庆 山东 海南 云南 贵州 安徽 吉林 青海 四川 内蒙古 湖北 宁夏 广西 新疆 上海 黑龙江 河南 山西 北京 广东 江苏 天津 湖南 陕西 福建 

In [ ]:
### 测试用

In [ ]:
import yaml

In [ ]:
with open('adidas_txt01.txt','r') as f:
    s=f.readlines()[0]
    js=yaml.load(s)
    print(js)

In [ ]:
ktt=[]
cc=0
for pkey,pval in province.items():
    prtt=['','','','']
    try:
        cc+=1
        pv=pval.replace('省','')
        prtt[0]=pv
        print(pv,cc,end=' ')
        res[pv]={}
        purl='https://www.adidas.com.cn/location/city.json?province={0}&_=1556377173555'.format(pv)
        cts=getOneUrl(purl)
        if cts==None:
            print('cts==None',pv,purl)
            cts='[]'
        citys=json.loads(cts)
        for c in citys:
            curl='https://www.adidas.com.cn/location/storelist.json?startPage=1&size=12&province={0}&city={1}&storeName=&_=1556376900755'.format(pv,c)
            crtxt=getOneUrl(curl)
            cjs=json.loads(crtxt)
            ktt.extend(cjs['storeMapResult'])
            cc+=1
            if cc%5==0:
                time.sleep(1)
            
    except:
        traceback.print_exc()
        print(prtt)

In [ ]:
print(len(ktt))
ktt[0]

In [ ]:
prov={1:'辽宁',2:'广东'}
cc=0
for pkey,pval in prov.items():
    prtt=['','','','']
    try:
        cc+=1
        pv=pval.replace('省','')
        prtt[0]=pv
        print(pv,cc,end=' ')
        res[pv]={}
        purl='https://www.adidas.com.cn/location/city.json?province={0}&_=1556377173555'.format(pv)
        cts=getOneUrl(purl)
        if cts==None:
            print('cts==None',pv,purl)
            cts='[]'
        citys=json.loads(cts)
        for c in citys:
            res[pv][c]={}
            prtt[1]=c
            sts=getStoreTypes(c) #sotretypes list str
            for s in sts:
                prtt[2]=s
                stotal=getSTotal(pv,c,s)
                res[pv][c][s]=stotal[0]
                klst.extend(stotal[1])
                prtt[3]=stotal
                #time.sleep(1)
            time.sleep(1)
            
    except:
        traceback.print_exc()
        print(prtt)

In [ ]:
prov={1:'辽宁'}
cc=0
for pv in ['辽宁']:
    prtt=['','','','']
    try:
        prtt[0]=pv
        print(pv,cc,end=' ')
        res[pv]={}
        for c in ['新民市']:
            res[pv][c]={}
            prtt[1]=c
            sts=getStoreTypes(c) #sotretypes list str
            for s in sts:
                prtt[2]=s
                stotal=getSTotal(pv,c,s)
                res[pv][c][s]=stotal[0]
                klst.extend(stotal[1])
                prtt[3]=stotal[0]
                #time.sleep(1)
            time.sleep(1)
            
    except:
        traceback.print_exc()
        print(prtt)

In [ ]:
ress

In [ ]:
#{'省':{'市':{'店':个数}}}

In [ ]:
#要坐标直接写 s['storeMapResult']   not  s['storeResult']  

In [ ]:
cts=requests.get('https://www.adidas.com.cn/location/city.json?province={0}&_=1556377173555'.format('江西')).text
citys=json.loads(cts)
citys

In [ ]:
sotretypes[0]['storeType']

In [22]:
stt=json.loads(requests.get(
                'https://www.adidas.com.cn/location/storelist.json?startPage=1&size=12&province={0}&city={1}&storeName=&types={2}&_=1556376900755'.format('江西','东乡县','运动表现店')).text)
stt

{'storeResult': [{'addressCN': '江西东乡县财富广场号东信百货1楼',
   'addressEN': '江西东乡县财富广场号东信百货1楼',
   'city': '东乡县',
   'id': 4955,
   'latitude': '28.242789',
   'longitude': '116.608582',
   'pOSNameCN': '东乡县财富广场东信百货运动表现店',
   'pOSNameEN': '东乡县财富广场东信百货运动表现店',
   'province': '江西',
   'storeType': '运动表现店',
   'storeTypeEN': 'Sport Performance',
   'telephone': '0794-4236288',
   'workHourCN': '',
   'workHourEn': ''}],
 'storeMapResult': [{'addressCN': '江西东乡县财富广场号东信百货1楼',
   'addressEN': '江西东乡县财富广场号东信百货1楼',
   'city': '东乡县',
   'id': 4955,
   'latitude': '28.242789',
   'longitude': '116.608582',
   'pOSNameCN': '东乡县财富广场东信百货运动表现店',
   'pOSNameEN': '东乡县财富广场东信百货运动表现店',
   'province': '江西',
   'storeType': '运动表现店',
   'storeTypeEN': 'Sport Performance',
   'telephone': '0794-4236288',
   'workHourCN': '',
   'workHourEn': ''}],
 'pageTotal': 1,
 'startPage': '1',
 'storeTotal': 1}